In [1]:
import tensorly as tl
import numpy as np
from numpy import linalg as la
from sympy import *

In [2]:
from tensorly.decomposition import parafac
from tensorly.decomposition import non_negative_parafac_hals
from tensorly.decomposition import non_negative_parafac

def check_rank(tensor, rank, non_neg=True, n=10, tol=0.0001, p=False):
    if non_neg:
        for k in range(n):
            weights, factors = non_negative_parafac_hals(tensor, n_iter_max=1000000, rank=rank, init='random')
            full = tl.cp_to_tensor((weights, factors))
            diff = (full - tensor) / tensor
            if p:
                # print('doing', k)
                # print(full)
                print(tl.max(abs(diff)))
                print(factors)
            if tl.max(abs(diff)) < tol:
                return True
    else:
        for k in range(n):
            weights, factors = parafac(tensor, n_iter_max=1000000, rank=rank)
            full = tl.cp_to_tensor((weights, factors))
            diff = (full - tensor) / tensor
            if p:
                print(tl.max(abs(diff)))
                print(factors)
            if tl.max(abs(diff)) < tol:
                return True
    return False


In [11]:
#generates random rank 3 tensors
def rank_tree():
    return (low_tensor() + low_tensor() + low_tensor()) 

#generates random rank 4+ (hopefully) tensors
def rank_for():
    return  (low_tensor() + low_tensor() + low_tensor() + low_tensor())


# generates rank 1, 2x2x3 tensors
def low_tensor():
    max = 300
    a = np.random.randint(1, max, size=3) * 0.1
    b = np.random.randint(1, max, size=2)* 0.1
    c = np.random.randint(1, max, size=2)* 0.1
    tens = tl.tensor(np.kron(np.kron(a, b), c).reshape(3, 2, 2)) * 1.0
    return tens

def low_tensor_spec():
    max = 300
    a = np.random.randint(1, max, size=2) * 0.1
    b = np.random.randint(1, max, size=3)* 0.1
    c = np.random.randint(1, max, size=3)* 0.1
    tens = tl.tensor(np.kron(np.kron(a, b), c).reshape(3, 2, 2 )) * 1.0
    return tens

def check(t):
    t1 = tl.tensor([t[0], t[1]])
    t2 = tl.tensor([t[1], t[2]])
    t2 = tl.tensor([t[1]+t[0], t[1]+t[2]])
    a1 = det(Matrix(t1[0]))
    a2 = det(Matrix(t1[1]))
    a3 = det(Matrix(t2[0]))
    a4 = det(Matrix(t2[1]))
    b1 = det(Matrix(t1[:,0]))
    b2 = det(Matrix(t1[:,1]))
    b3 = det(Matrix(t2[:,0]))
    b4 = det(Matrix(t2[:,1]))
    c1 = det(Matrix(t1[:,:,0]))
    c2 = det(Matrix(t1[:,:,1]))
    c3 = det(Matrix(t2[:,:,0]))
    c4 = det(Matrix(t2[:,:,1]))

    #a5 = det(Matrix(t2[0]))
    #a6 = det(Matrix(t2[1]))

    #b5 = det(Matrix(t2[:,0]))
    #b6 = det(Matrix(t2[:,1]))

    #c5 = det(Matrix(t2[:,:,0]))
    #c6 = det(Matrix(t2[:,:,1]))
    return sgn([a1, a2,a3,a4]) or sgn([b1,b2,b3,b4]) or sgn([c1,c2,c3,c4])

def sgn(a):
    t = 0
    ab = 0
    for a_i in a:
        t+= abs(a_i)
        ab += a_i
    return t == abs(ab)

def check_r3(t):
    a1 = det(Matrix(t[0]))
    a2 = det(Matrix(t[1]))
    b1 = det(Matrix(t[:,0]))
    b2 = det(Matrix(t[:,1]))
    c1 = det(Matrix(t[:,:,0]))
    c2 = det(Matrix(t[:,:,1]))
    return sgn([a1,a2]) or sgn([b1,b2]) or sgn([c1,c2])

def check_r2(t):
    a1 = det(Matrix(t[0]))
    a2 = det(Matrix(t[1]))
    b1 = det(Matrix(t[:,0]))
    b2 = det(Matrix(t[:,1]))
    c1 = det(Matrix(t[:,:,0]))
    c2 = det(Matrix(t[:,:,1]))
    d1 = ineq(a1,a2,b1,b2,c1,c2,ge,ge,ge)
    d2 = ineq(a1,a2,b1,b2,c1,c2,le,le,ge)
    d3 = ineq(a1,a2,b1,b2,c1,c2,le,ge,le)
    d4 = ineq(a1,a2,b1,b2,c1,c2,ge,le,le)
    supermod = d1 or d2 or d3 or d4
    return supermod

def ineq(a1,a2,b1,b2,c1,c2,f1,f2,f3):
    t1 = f1(a1,0) and f1(a2,0)
    t2 = f2(b1,0) and f2(b2,0)
    t3 = f3(c1,0) and f3(c2,0)
    return t1 and t2 and t3
    
def ge(a1,a2):
    return a1 >= a2

def le(a1,a2):
    return a1 <= a2

def check_r4(t):
    a1 = not check_r3(tl.tensor([t[0],t[1]]))
    a2 = not check_r3(tl.tensor([t[0],t[2]]))
    a3 = not check_r3(tl.tensor([t[1],t[2]]))
    return a1 or a2 or a3

def check_sub(t, i, j):
    temp = np.copy(t)
    temp[0][i][j] = 0
    temp[1][i][j] = 0
    temp[2][i][j] = 0
    t1 = tl.tensor([temp[0],temp[1]])
    t2 = tl.tensor([temp[0],temp[2]])
    t3 = tl.tensor([temp[1],temp[2]])
    return check_r2(t1) and check_r2(t2) and check_r2(t3)
    

In [15]:
print(check_r2(np.random.randint(1, 200, size=(2,2,2))))

True


In [21]:
Matrix([[Matrix(tens[:,:,0]),Matrix(tens[:,:,1])]])

Matrix([
[ 15053.22, 15797.052,  8609.416,    8589.0],
[ 6655.732,  7118.406, 17119.988, 17059.744],
[16343.428, 17273.598,  9322.256,  9758.492]])

In [19]:
Matrix([[tens[:,:,0],tens[:,:,1]]])

Matrix([[[[15053.2200000000, 15797.0520000000], [6655.73200000000, 7118.40600000000], [16343.4280000000, 17273.5980000000]], [[8609.41600000000, 8589.00000000000], [17119.9880000000, 17059.7440000000], [9322.25600000000, 9758.49200000000]]]])

In [59]:
t1 = tl.tensor([tens[0], tens[1]])
t2 = tl.tensor([tens[1], tens[2]])
a1 = det(Matrix(t1[0]))
a2 = det(Matrix(t1[1]))
a3 = det(Matrix(t2[0]))
a4 = det(Matrix(t2[1]))
b1 = det(Matrix(t1[:,0]))
b2 = det(Matrix(t1[:,1]))
b3 = det(Matrix(t2[:,0]))
b4 = det(Matrix(t2[:,1]))
c1 = det(Matrix(t1[:,:,0]))
c2 = det(Matrix(t1[:,:,1]))
c3 = det(Matrix(t2[:,:,0]))
c4 = det(Matrix(t2[:,:,1]))
print(check(tens))

True


In [27]:
counter = 0
tensor_r4 = []
for i in range(2000):
    t = rank_for()
    tens = tl.tensor(np.random.randint(100, 2000000, size=(3,2,2))) * 0.01
    if check_r4(tens):
        tensor_r4.append(tens)
        if check_sub(tens, 0,0) or check_sub(tens, 1,0) or check_sub(tens, 0,1) or check_sub(tens, 1,1):
            counter += 1
print(counter)

[[[ 4666.92 16869.7 ]
  [19066.2   6344.81]]

 [[17891.12 12077.95]
  [ 9340.69 16540.91]]

 [[  902.53 18879.51]
  [10250.98  7614.94]]]


In [28]:
print(counter)

0


In [18]:
count = 0
for i in range(100):
    if check_sub(tensor_r4[i+20], 0, 0):
        count += 1
        
print(count)

0


In [26]:
count = 0
for i in range(100): 
    tens = tensor_r4[i+200]
    if check_sub(tens, 0,0) or check_sub(tens, 1,0) or check_sub(tens, 0,1) or check_sub(tens, 1,1):
        count += 1
print(count)

0


In [253]:
print(tensor_r4[1002])

[[[    0.    6281.91]
  [    0.   17149.45]]

 [[    0.   14292.4 ]
  [    0.   12637.41]]

 [[    0.    1534.81]
  [    0.    6700.4 ]]]


In [235]:
    M = Matrix([[Matrix(tens[:,1]),Matrix(tens[:,0])]])
    M1 = Matrix([[M[:,0],M[:,1],M[:,2]]])
    M2 = Matrix([[M[:,0],M[:,3],M[:,2]]])
    M3 = Matrix([[M[:,3],M[:,2],M[:,1]]])
    M4 = Matrix([[M[:,3],M[:,1],M[:,0]]])

-35991306622.1029


In [230]:
M2.rref()[0]

Matrix([
[1, 0, 0],
[0, 1, 0],
[0, 0, 1]])

In [231]:
M3.rref()[0]

Matrix([
[1, 0, 0],
[0, 1, 0],
[0, 0, 1]])

In [232]:
M4.rref()[0]

Matrix([
[1, 0, 0],
[0, 1, 0],
[0, 0, 1]])

In [184]:
K = M4.rref()[0][:,3]
K

Matrix([
[0.0555975569303306],
[ 0.939245855062972],
[-0.272028900972627]])

In [187]:
np.array(K)

array([[0.0555975569303306],
       [0.939245855062972],
       [-0.272028900972627]], dtype=object)

In [218]:
check_rank(tens, 3, p = True)

1.1010236849202595
[array([[15662.09260276, 12584.21401129,   556.23305632],
       [ 8988.28341608, 14894.42948998,     0.        ],
       [ 9526.58124322, 15044.73420337,   193.2194049 ]]), array([[ 0.74081396,  0.49884142, 11.05972315],
       [ 0.80872382,  0.83760546,  0.        ]]), array([[0.        , 0.80629132, 1.08818613],
       [1.04173683, 0.        , 0.        ]])]
0.3051047525407385
[array([[14495.48138099,  1344.07513256, 10686.53607437],
       [14440.25495165,     0.        ,  8585.13803643],
       [ 5817.21460626,  1195.54557485,  9864.81752175]]), array([[0.94499894, 0.        , 1.88762579],
       [0.30225456, 8.5077478 , 2.01641285]]), array([[0.        , 0.        , 0.48234794],
       [0.82577357, 0.89961888, 0.        ]])]
0.30510505968415463
[array([[39883.25531396, 14087.35238726, 27935.43611174],
       [ 2414.92624591, 11317.22539777, 28460.82202871],
       [32632.70730546, 13004.14013841,  9501.70785521]]), array([[0.        , 0.65997052, 0.5526391 ],
 

False